In [12]:
# Import necessary libraries
import pandas as pd
import numpy as np
import zipfile
import os
import re
from IPython.display import display

# Try to load the CSV file
df = pd.read_csv('salaries.csv')
print("Data loaded successfully with", len(df), "records")
    
# Show the first few rows to verify
print("\nFirst 10 rows of data:")
display(df.head(10))

# Clean the data by filling missing values with 0
df.fillna(0, inplace=True)
print("\nData cleaned (missing values filled with 0)")
def get_employee_details(name, exact_match=True):
    """
    Find employee by name and return their details
    
    Parameters: 
    - name: The name to search for
    - exact_match: Whether to match the full name (True) or allow partial matches (False)
    
    Returns:
    Dictionary of employee details or error message
    """
    try:
        # Search for employee
        if exact_match:
            # Match exact name (case insensitive)
            employee = df[df['EmployeeName'].str.lower() == name.lower()]
        else:
            # Allow partial matches
            employee = df[df['EmployeeName'].str.lower().str.contains(name.lower())]
            # Check if found
        if len(employee) == 0:
            return {"error": "No employee found with that name"}
        
        # Convert to dictionary and return first match
        return employee.iloc[0].to_dict()
        
    except Exception as e:
        return {"error": f"Search failed: {str(e)}"}

# Sample messy salary data
data = {
    'EmployeeName': ['  john smith  ', 'JANE DOE', 'Robert Johnson', 'emily DAVIS', 'MICHAEL brown', 'NULL', '12345'],
    'JobTitle': ['manager', 'SENIOR DEVELOPER', 'Data Analyst', 'HR MANAGER', 'devops Engineer', '', 'Admin'],
    'BasePay': ['75,000', '$65000', '60000', '55K', '68,000.50', 'N/A', 'abc'],
    'OvertimePay': ['5,000', '2000', '0', '1k', '3,500.25', '-', 'xyz'],
    'Benefits': ['15%', '12000', '10000', '9,000', '13K', '', '10,000']
}
df = pd.DataFrame(data)

# Test the function
print("\nTesting employee search:")
print("Exact match for 'Jane Doe':", get_employee_details("Jane Doe"))
print("Partial match for 'john:", get_employee_details("john", exact_match=False))
print("Non-existent employee:", get_employee_details("Nobody Here"))

# Load data (replace with your actual data loading)
data = {
    'EmployeeName': ['John Smith', 'Jane Doe', 'John Smith', 'Emily Davis', 'Jane Doe'],
    'JobTitle': ['Manager', 'Developer', 'Manager', 'Designer', 'Developer'],
    'BasePay': [75000, 65000, 80000, 55000, 70000],
    'OvertimePay': [5000, 2000, 6000, 1000, 3000],
    'Benefits': [15000, 12000, 16000, 9000, 13000]
}
df = pd.DataFrame(data)

def clean_text_columns(series):
    """Clean and standardize text columns (names, job titles)"""
    return (
        series
        .astype(str)
        .str.strip()
        .str.title()  # Convert to Title Case
        .str.replace(r"[^a-zA-Z\s\-/]", "", regex=True)  # Remove special chars (keep hyphens/slashes)
        .str.replace(r"\s+", " ", regex=True)  # Collapse multiple spaces
        .replace(["Null", "Nan", "None", ""], "Unknown")
    )

def clean_currency(series):
    """Clean currency/numeric columns"""
    return (
        series
        .astype(str)
        .str.strip()
        .str.replace(r"[$,%kK]", "", regex=True)  # Remove $,% and k/K
        .str.replace(",", "")  # Remove thousands separators
        .replace(["N/A", "NA", "-", ""], "0")  # Handle missing values
        .apply(lambda x: re.sub(r"[^\d.]", "", x))  # Remove all non-numeric except decimal
        .astype(float)
    )
# Clean text columns
df['EmployeeName'] = clean_text_columns(df['EmployeeName'])
df['JobTitle'] = clean_text_columns(df['JobTitle'])

# Clean numeric columns
numeric_cols = ['BasePay', 'OvertimePay', 'Benefits']
df[numeric_cols] = df[numeric_cols].apply(clean_currency)

# Final validation
print("Cleaned Data Overview:")
print(df.head())
print("\nData Types:")
print(df.dtypes)
print("\nSummary Statistics:")
print(df[numeric_cols].describe())

# Check for remaining anomalies
text_anomalies = df[~df['EmployeeName'].str.match(r'^[A-Z][a-z]+(?:[ \-][A-Z][a-z]+)*$')]
num_anomalies = df[df[numeric_cols].lt(0).any(axis=1)]

if not text_anomalies.empty:
    print("\nText anomalies found:")
    print(text_anomalies)
if not num_anomalies.empty:
    print("\nNegative values found in numeric columns:")
    print(num_anomalies)

# Handle duplicate names by adding identifiers
df['EmployeeName'] = df.groupby('EmployeeName')['EmployeeName'].transform(
    lambda x: x + ' (' + (x.groupby(x).cumcount() + 1).astype(str) + ')' 
    if x.duplicated().any() else x
)

# Convert to dictionary with unique index
employee_dict = df.set_index('EmployeeName').to_dict(orient='index')

# Analysis functions
def get_average_pay_by_title(emp_dict):
    """Calculate average pay metrics by job title"""
    title_stats = {}
    for emp, details in emp_dict.items():
        title = details['JobTitle']
        if title not in title_stats:
            title_stats[title] = {
                'base_pays': [],
                'overtime_pays': [],
                'benefits': [],
                'count': 0
            }
        title_stats[title]['base_pays'].append(details['BasePay'])
        title_stats[title]['overtime_pays'].append(details['OvertimePay'])
        title_stats[title]['benefits'].append(details['Benefits'])
        title_stats[title]['count'] += 1

# Calculate averages
    avg_pay = {}
    for title, stats in title_stats.items():
        avg_pay[title] = {
            'avg_base': np.mean(stats['base_pays']),
            'avg_overtime': np.mean(stats['overtime_pays']),
            'avg_benefits': np.mean(stats['benefits']),
            'employee_count': stats['count']
        }
    return avg_pay

def find_highest_paid(emp_dict):
    """Find employee with highest total compensation"""
    max_pay = 0
    top_employee = None
    for emp, details in emp_dict.items():
        total = details['BasePay'] + details['OvertimePay'] + details['Benefits']
        if total > max_pay:
            max_pay = total
            top_employee = (emp, details['JobTitle'], total)
    return top_employee

# Run analyses
print("\nEmployee Dictionary Sample:")
print({k: employee_dict[k] for k in list(employee_dict)[:5]})  # Print first 5 entries

avg_pay = get_average_pay_by_title(employee_dict)
print("\nAverage Pay by Title:")
for title, stats in avg_pay.items():
    print(f"\n{title} ({stats['employee_count']} employees):")
    print(f"  Avg Base Pay: ${stats['avg_base']:,.2f}")
    print(f"  Avg Overtime: ${stats['avg_overtime']:,.2f}")
    print(f"  Avg Benefits: ${stats['avg_benefits']:,.2f}")

top_earner = find_highest_paid(employee_dict)
print(f"\nHighest Paid Employee: {top_earner[0]} ({top_earner[1]}) - ${top_earner[2]:,.2f}")

#Export to JSON
import json
with open('employee_analysis.json', 'w') as f:
    json.dump({
        'employee_data': employee_dict,
        'average_pay': avg_pay,
        'top_earner': top_earner
    }, f, indent=2)
print("\nAnalysis saved to employee_analysis.json")
def compare_employees(name1, name2):
    """
    Compare two employees' total compensation
    
    Parameters:
    - name1: First employee name
    - name2: Second employee name
    
    Returns:
    Comparison result or error message
    """
# Look up both employees
    emp1 = get_employee_details(name1)
    emp2 = get_employee_details(name2)
# Check for errors
    if 'error' in emp1:
            return f"Error with first employee: {emp1['error']}"
    if 'error' in emp2:
            return f"Error with second employee: {emp2['error']}"
        
# Get total compensation (base + benefits)
    total1 = emp1['TotalPayBenefits']
    total2 = emp2['TotalPayBenefits']
        
# Prevent division by zero
    if total2 == 0:
            return "Cannot compare - second employee has no compensation data"
        
# Calculate comparison metrics
    ratio = total1 / total2
    difference = total1 - total2
    return (f"Comparison between {name1} (${total1:,.2f}) and {name2} (${total2:,.2f}):\n"
                f"  Ratio: {ratio:.2f}\n"
                f"  Difference: ${difference:,.2f}")

# Test the function
print("\nTesting employee comparison:")
print(compare_employees("Jane Doe", "John Smith"))  # Valid
print(compare_employees("Jane Doe", "Unknown"))     # Error case

def export_employee(name, folder="Employee Profiles"):
    """
    Export an employee's data to a zipped CSV file
    
    Parameters:
    - name: Employee name to export
    - folder: Folder name to use for export
    
    Returns:
    Success message or error
    """
    try:
        # Find the employee (allowing partial matches)
        emp_data = get_employee_details(name, exact_match=False)
        if 'error' in emp_data:
            return emp_data['error']
        
        # Create folder if needed
        os.makedirs(folder, exist_ok=True)
        # Create filename (replace spaces with underscores)
        filename = f"{name.replace(' ', '_')}_profile.csv"
        filepath = os.path.join(folder, filename)
        zippath = f"{folder}.zip"
        
        # Save to CSV
        pd.DataFrame([emp_data]).to_csv(filepath, index=False)
        
        # Create zip file
        with zipfile.ZipFile(zippath, 'w') as zipf:
            zipf.write(filepath, arcname=filename)
        
        # Clean up
        os.remove(filepath)
        
        return f"Successfully exported {name}'s data to {zippath}"
    
    except Exception as e:
        return f"Export failed: {str(e)}"
    
    # Test exporting
print("\nTesting employee export:")
print(export_employee("Jane Doe"))
print(export_employee("john"))  # Partial match





Data loaded successfully with 312882 records

First 10 rows of data:


C:\Users\mdoba\AppData\Local\Temp\ipykernel_33104\3805607594.py:10: DtypeWarning: Columns (2,3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('salaries.csv')


,EmployeeName,JobTitle,BasePay,OvertimePay,OtherPay,Benefits,TotalPay,TotalPayBenefits,Year
0,NATHANIEL FORD,GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY,167411.18,0.0,400184.25,Not Provided,567595.43,567595.43,2011
1,GARY JIMENEZ,CAPTAIN III (POLICE DEPARTMENT),155966.02,245131.88,137811.38,Not Provided,538909.28,538909.28,2011
2,ALBERT PARDINI,CAPTAIN III (POLICE DEPARTMENT),212739.13,106088.18,16452.6,Not Provided,335279.91,335279.91,2011
3,CHRISTOPHER CHONG,WIRE ROPE CABLE MAINTENANCE MECHANIC,77916.0,56120.71,198306.9,Not Provided,332343.61,332343.61,2011
4,PATRICK GARDNER,"DEPUTY CHIEF OF DEPARTMENT,(FIRE DEPARTMENT)",134401.6,9737.0,182234.59,Not Provided,326373.19,326373.19,2011
5,DAVID SULLIVAN,ASSISTANT DEPUTY CHIEF II,118602.0,8601.0,189082.74,Not Provided,316285.74,316285.74,2011
6,ALSON LEE,"BATTALION CHIEF, (FIRE DEPARTMENT)",92492.01,89062.9,134426.14,Not Provided,315981.05,315981.05,2011
7,DAVID KUSHNER,DEPUTY DIRECTOR OF INVESTMENTS,256576.96,0.0,51322.5,Not Provided,307899.46,307899.46,2011
8,MICHAEL MORRIS,"BATTALION CHIEF, (FIRE DEPARTMENT)",176932.64,86362.68,40132.23,Not Provided,303427.55,303427.55,2011
9,JOANNE HAYES-WHITE,"CHIEF OF DEPARTMENT, (FIRE DEPARTMENT)",285262.0,0.0,17115.73,Not Provided,302377.73,302377.73,2011



Data cleaned (missing values filled with 0)

Testing employee search:
Exact match for 'Jane Doe': {'EmployeeName': 'JANE DOE', 'JobTitle': 'SENIOR DEVELOPER', 'BasePay': '$65000', 'OvertimePay': '2000', 'Benefits': '12000'}
Partial match for 'john: {'EmployeeName': '  john smith  ', 'JobTitle': 'manager', 'BasePay': '75,000', 'OvertimePay': '5,000', 'Benefits': '15%'}
Non-existent employee: {'error': 'No employee found with that name'}
Cleaned Data Overview:
  EmployeeName   JobTitle  BasePay  OvertimePay  Benefits
0   John Smith    Manager  75000.0       5000.0   15000.0
1     Jane Doe  Developer  65000.0       2000.0   12000.0
2   John Smith    Manager  80000.0       6000.0   16000.0
3  Emily Davis   Designer  55000.0       1000.0    9000.0
4     Jane Doe  Developer  70000.0       3000.0   13000.0

Data Types:
EmployeeName     object
JobTitle         object
BasePay         float64
OvertimePay     float64
Benefits        float64
dtype: object

Summary Statistics:
            BasePay 

In [13]:
# Create an R script file
r_code = """# Employee Data Viewer
# Unzips and displays employee data

# Unzip the file
unzip("Employee Profiles.zip", exdir = "temp_employee_data")

# Find the CSV file
csv_file <- list.files("temp_employee_data", pattern = "\\.csv$", full.names = TRUE)[1]

# Read and display data
employee_data <- read.csv(csv_file)
print("Employee Profile:")
print(employee_data)

# Clean up
unlink("temp_employee_data", recursive = TRUE)
"""
# Save to file
with open("view_employee.R", "w") as f:
    f.write(r_code)

print("\nCreated R script 'view_employee.R' to unzip and view employee data")



Created R script 'view_employee.R' to unzip and view employee data
